Classement général

In [1]:
import pandas as pd

# === Lecture du fichier avec en-tête ===
df = pd.read_csv("C:\\1_Programmes Python\\Statistiques\\Classement Elo foot\\New version\\Résultats\\all_results\\resultats_1997_2026.csv")

# convertir les buts en entiers (sécurité si certains sont lus en str)
df["buts_dom"] = pd.to_numeric(df["buts_dom"], errors="coerce").fillna(0).astype(int)
df["buts_ext"] = pd.to_numeric(df["buts_ext"], errors="coerce").fillna(0).astype(int)

# === Barème par compétition ===
points_system = {
    "ligue-1": {"W": 9, "D": 3, "L": 0},
    "ligue-2": {"W": 6, "D": 2, "L": 0},
    "national": {"W": 3, "D": 1, "L": 0}
}

# === Stats par équipe ===
stats = {}

def update_stats(equipe, comp, scored, conceded, result):
    if equipe not in stats:
        stats[equipe] = {"points": 0,
            "matches": 0, "wins": 0, "draws": 0, "losses": 0,
            "goals_for": 0, "goals_against": 0, "goal_diff": 0
        }
    stats[equipe]["matches"] += 1
    stats[equipe]["goals_for"] += scored
    stats[equipe]["goals_against"] += conceded
    stats[equipe]["goal_diff"] = stats[equipe]["goals_for"] - stats[equipe]["goals_against"]

    if result == "W":
        stats[equipe]["wins"] += 1
    elif result == "D":
        stats[equipe]["draws"] += 1
    else:
        stats[equipe]["losses"] += 1

    # attribution des points
    if comp in points_system:  # sécurité si comp inconnue
        stats[equipe]["points"] += points_system[comp][result]

# === Parcourir les matchs ===
for _, row in df.iterrows():
    comp = row["competition"]
    dom, ext = row["equipe_dom"], row["equipe_ext"]
    buts_dom, buts_ext = row["buts_dom"], row["buts_ext"]

    if buts_dom > buts_ext:
        res_dom, res_ext = "W", "L"
    elif buts_dom < buts_ext:
        res_dom, res_ext = "L", "W"
    else:
        res_dom, res_ext = "D", "D"

    update_stats(dom, comp, buts_dom, buts_ext, res_dom)
    update_stats(ext, comp, buts_ext, buts_dom, res_ext)

# === Convertir en DataFrame ===
classement = pd.DataFrame.from_dict(stats, orient="index")

# trier : points > diff > buts pour
classement = classement.sort_values(by=["points", "goal_diff", "goals_for"], ascending=[False, False, False])

# ajouter le rang et renommer les colonnes
classement.reset_index(inplace=True)
classement.rename(columns={"index": "équipe"}, inplace=True)
classement.insert(0, "rang", range(1, len(classement) + 1))

# === Sauvegarder ===
classement.to_csv("classement_global.csv", index=False)

print(classement.head(20))  # affiche les 20 premiers



classement.to_csv("C:\\1_Programmes Python\\Statistiques\\Classement Elo foot\\New version\\Data Visualization\\classement_global.csv",index=False)
classement.to_excel("C:\\1_Programmes Python\\Statistiques\\Classement Elo foot\\New version\\Data Visualization\\classement_global.xlsx", index=False, engine="openpyxl")



    rang               équipe  points  matches  wins  draws  losses  \
0      1  Paris Saint-Germain    5838     1034   565    251     218   
1      2   Olympique Lyonnais    5682     1035   549    247     239   
2      3  Olympique Marseille    5187     1035   485    274     276   
3      4            AS Monaco    5029     1035   479    282     274   
4      5            Lille OSC    5002     1051   478    304     269   
5      6   Girondins Bordeaux    4419      996   405    298     293   
6      7        Stade Rennais    4416     1035   401    269     365   
7      8             OGC Nice    4241     1059   392    324     343   
8      9              RC Lens    4094     1035   409    307     319   
9     10     AS Saint-Étienne    4004     1058   393    300     365   
10    11      Montpellier HSC    3755     1042   361    286     395   
11    12           AJ Auxerre    3746     1039   380    300     359   
12    13            FC Nantes    3732     1035   350    288     397   
13    

Avec les logs

In [2]:
import pandas as pd

# === Lecture du fichier ===
df = pd.read_csv("C:\\1_Programmes Python\\Statistiques\\Classement Elo foot\\New version\\Résultats\\all_results\\resultats_1997_2026.csv")

# convertir les buts en entiers
df["buts_dom"] = pd.to_numeric(df["buts_dom"], errors="coerce").fillna(0).astype(int)
df["buts_ext"] = pd.to_numeric(df["buts_ext"], errors="coerce").fillna(0).astype(int)

# === Barème par compétition ===
points_system = {
    "ligue-1": {"W": 9, "D": 3, "L": 0},
    "ligue-2": {"W": 6, "D": 2, "L": 0},
    "national": {"W": 3, "D": 1, "L": 0}
}

# === Stats par équipe ===
stats = {}

def update_stats(equipe, comp, scored, conceded, result):
    if equipe not in stats:
        stats[equipe] = {
            "points": 0,
            "matches": 0, "wins": 0, "draws": 0, "losses": 0,
            "goals_for": 0, "goals_against": 0, "goal_diff": 0
        }
    stats[equipe]["matches"] += 1
    stats[equipe]["goals_for"] += scored
    stats[equipe]["goals_against"] += conceded
    stats[equipe]["goal_diff"] = stats[equipe]["goals_for"] - stats[equipe]["goals_against"]

    if result == "W":
        stats[equipe]["wins"] += 1
    elif result == "D":
        stats[equipe]["draws"] += 1
    else:
        stats[equipe]["losses"] += 1

    # attribution des points
    if comp in points_system:
        stats[equipe]["points"] += points_system[comp][result]

# === Préparer toutes les équipes pour remplir 0 par défaut ===
all_teams = sorted(set(df["equipe_dom"]).union(df["equipe_ext"]))

# === Tri chronologique ===
df = df.sort_values("date")

# === Écriture directe dans le CSV avec 0 remplacé par '' ===
with open("C:\\1_Programmes Python\\Statistiques\\Classement Elo foot\\New version\\Data Visualization\\log_points.csv", "w", encoding="utf-8") as f:
    # Écriture de l'en-tête
    f.write("date," + ",".join(all_teams) + "\n")

    # Boucle sur les matchs
    for _, row in df.iterrows():
        comp = row["competition"]
        dom, ext = row["equipe_dom"], row["equipe_ext"]
        buts_dom, buts_ext = row["buts_dom"], row["buts_ext"]

        # Déterminer le résultat du match
        if buts_dom > buts_ext:
            res_dom, res_ext = "W", "L"
        elif buts_dom < buts_ext:
            res_dom, res_ext = "L", "W"
        else:
            res_dom = res_ext = "D"

        # Mettre à jour les stats
        update_stats(dom, comp, buts_dom, buts_ext, res_dom)
        update_stats(ext, comp, buts_ext, buts_dom, res_ext)

        # Préparer la ligne CSV : remplacer 0 par blanc
        line = [row["date"]] + [
            "" if stats.get(team, {"points":0})["points"] == 0 else str(stats[team]["points"])
            for team in all_teams
        ]
        f.write(",".join(line) + "\n")

print("CSV log_points.csv généré avec des blancs à la place des 0.")


CSV log_points.csv généré avec des blancs à la place des 0.
